# Recommandation de films par similarité utilisateurs

L’objectif est d’identifier, pour chaque utilisateur, les films qu’il appréciera le plus, en se basant sur les utilisateurs possédant des goûts similaires.

On construit un système User-Based Collaborative Filtering avec un KNN appliqué dans l’espace des préférences utilisateur.
Plus précisément, chaque utilisateur sera représenté par un vecteur de notes dans une grande matrice de dimension utilisateurs × films. Le KNN sera ensuite utilisé pour trouver, dans cet espace, les utilisateurs dont les profils de notation sont les plus proches.

Ce système permet :
- Recommandation personnalisée (comme Netflix, Amazon Prime…)
- Compréhension des profils d’utilisateurs
- Suggestion de films “du même style” appréciés par des utilisateurs voisins

# 1. Données

Les données utilisées pour ce projet proviennent du jeu de données MovieLens, qui contient des évaluations de films par des utilisateurs. Nous utiliserons la version "MovieLens 100K", qui comprend 100 000 évaluations de films faites par 943 utilisateurs sur 1682 films.

Le jeu de données est disponible à l'adresse suivante : [MovieLens 100K Dataset](https://grouplens.org/datasets/movielens/100k/).

In [1]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

In [3]:
# Charger le dataset
ratings = pd.read_csv("u.data", sep="\t", names=["userId", "movieId", "rating", "timestamp"])

ratings.head()

,userId,movieId,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
movies = pd.read_csv("u.item", sep="|", encoding="latin-1",
                     names=["movieId", "title", "releaseDate", "videoReleaseDate", "IMDbURL", "genreUnknow", "genreAction", "genreAdventure", "genreAnimation", "genreChildren", "genreComedy", "genreCrime", "genreDocumentary", "genreDrama", "genreFantasy", "genreFilmNoir", "genreHorror", "genreMusical", "genreMystery", "genreRomance", "genreSciFi", "genreThriller", "genreWar", "genreWestern"],
                    )

movies.head()

,movieId,title,releaseDate,videoReleaseDate,IMDbURL,genreUnknow,genreAction,genreAdventure,genreAnimation,genreChildren,...,genreFantasy,genreFilmNoir,genreHorror,genreMusical,genreMystery,genreRomance,genreSciFi,genreThriller,genreWar,genreWestern
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


Les 19 dernières colonnes du fichier "u.item" correspondent aux genres des films (1 si le film appartient à ce genre, 0 sinon) : inconnu, action, aventure, animation, enfants, comédie, crime, documentaire, drame, fantasy, film noir, horreur, musical, mystère, romance, science-fiction, thriller, guerre, western.

## Aperçu des données

In [5]:
print("Taille de ratings :", ratings.shape)
print("Taille de movies  :", movies.shape)

n_users = ratings["userId"].nunique()
n_movies_rated = ratings["movieId"].nunique()

print(f"Nombre d'utilisateurs distincts dans ratings : {n_users}")
print(f"Nombre de films distincts notés            : {n_movies_rated}")

print("\nAperçu des notes :")
print(ratings["rating"].describe())

print("\nRépartition des notes (counts) :")
print(ratings["rating"].value_counts().sort_index())

Taille de ratings : (100000, 4)
Taille de movies  : (1682, 24)
Nombre d'utilisateurs distincts dans ratings : 943
Nombre de films distincts notés            : 1682

Aperçu des notes :
count    100000.000000
mean          3.529860
std           1.125674
min           1.000000
25%           3.000000
50%           4.000000
75%           4.000000
max           5.000000
Name: rating, dtype: float64

Répartition des notes (counts) :
rating
1     6110
2    11370
3    27145
4    34174
5    21201
Name: count, dtype: int64


## Fusion des notes et des films

In [6]:
# Merge ratings avec movies pour obtenir les titres des films
ratings = ratings.join(movies[["title"]], on="movieId")
ratings.head()

,userId,movieId,rating,timestamp,title
0,196,242,3,881250949,Jungle2Jungle (1997)
1,186,302,3,891717742,Ulee's Gold (1997)
2,22,377,1,878887116,Miracle on 34th Street (1994)
3,244,51,2,880606923,"Madness of King George, The (1994)"
4,166,346,1,886397596,Wag the Dog (1997)


À ce stade, on a :
- Une table data propre, qui contient ce que l’utilisateur a noté + quel film c’est.
- C’est cette table qui servira de base pour :
  - Construire la matrice utilisateur × film,
  - Puis entraîner notre KNN user-based.

# 2. Construction de la matrice utilisateur × film

On va créer une matrice où :
- Chaque ligne = un utilisateur
- Chaque colonne = un film
- Chaque case = la note donnée (ou NaN si pas noté)

In [7]:
# Créer une matrice utilisateur-film (user-item matrix)
# Rows: userId, Columns: movieId, Values: rating
df_with_nan = ratings.pivot_table(index='userId', columns='title', values='rating')

## préparer la matrice pour KNN

Le KNN de sklearn attend une matrice **numérique** complète (pas de NaN).
On va donc :
1. Remplacer les NaN par 0 (interprété comme “pas de note”, car les notes commencent à 1).
2. Convertir en numpy pour l’entraînement.

In [8]:
# Remplir les valeurs manquantes avec 0 (KNN ne peut pas gérer les NaN)
df = df_with_nan.fillna(0)
df.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,5.0,5.0,0.0,0.0,1.0,4.0,0.0,0.0,...,0.0,0.0,0.0,5.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0


# 3. Le modèle KNN

On utilise `NearestNeighbors` de scikit-learn avec `metric="cosine"` : mesure de similarité adaptée aux vecteurs de notes (on regarde la forme du profil plus que le niveau absolu).

In [10]:
# Créer le modèle KNN
clf = NearestNeighbors(metric='cosine')
clf.fit(df)

,"n_neighbors n_neighbors: int, default=5Number of neighbors to use by default for :meth:`kneighbors` queries.",5
,"radius radius: float, default=1.0Range of parameter space to use by default for :meth:`radius_neighbors`queries.",1.0
,"algorithm algorithm: {'auto', 'ball_tree', 'kd_tree', 'brute'}, default='auto'Algorithm used to compute the nearest neighbors:- 'ball_tree' will use :class:`BallTree`- 'kd_tree' will use :class:`KDTree`- 'brute' will use a brute-force search.- 'auto' will attempt to decide the most appropriate algorithm based on the values passed to :meth:`fit` method.Note: fitting on sparse input will override the setting ofthis parameter, using brute force.",'auto'
,"leaf_size leaf_size: int, default=30Leaf size passed to BallTree or KDTree. This can affect thespeed of the construction and query, as well as the memoryrequired to store the tree. The optimal value depends on thenature of the problem.",30
,"metric metric: str or callable, default='minkowski'Metric to use for distance computation. Default is ""minkowski"", whichresults in the standard Euclidean distance when p = 2. See thedocumentation of `scipy.spatial.distance`_ andthe metrics listed in:class:`~sklearn.metrics.pairwise.distance_metrics` for valid metricvalues.If metric is ""precomputed"", X is assumed to be a distance matrix andmust be square during fit. X may be a :term:`sparse graph`, in whichcase only ""nonzero"" elements may be considered neighbors.If metric is a callable function, it takes two arrays representing 1Dvectors as inputs and must return one value indicating the distancebetween those vectors. This works for Scipy's metrics, but is lessefficient than passing the metric name as a string.",'cosine'
,"p p: float (positive), default=2Parameter for the Minkowski metric fromsklearn.metrics.pairwise.pairwise_distances. When p = 1, this isequivalent to using manhattan_distance (l1), and euclidean_distance(l2) for p = 2. For arbitrary p, minkowski_distance (l_p) is used.",2
,"metric_params metric_params: dict, default=NoneAdditional keyword arguments for the metric function.",None
,"n_jobs n_jobs: int, default=NoneThe number of parallel jobs to run for neighbors search.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None


## Test avec un utilisateur

In [11]:
# Test avec un utilisateur spécifique
user_id = 42

user_index = df.index.get_loc(user_id)

distances, indices = clf.kneighbors(df.iloc[user_index, :].values.reshape(1, -1), n_neighbors=6)

print("Indices :", indices)
print("Distances :", distances)

neighbor_indices = indices.flatten()
neighbor_user_ids = df.index[neighbor_indices]
print("User IDs des voisins les plus proches :", neighbor_user_ids)


Indices : [[ 41 576 863 310 289 617]]
Distances : [[0.         0.4364331  0.43679242 0.44224413 0.45289566 0.45340452]]
User IDs des voisins les plus proches : Index([42, 577, 864, 311, 290, 618], dtype='int64', name='userId')


/Users/bilal/Documents/Projects/ML_Pro_projects/.venv/lib/python3.14/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


# 4. Générer des recommandations avec KNN (User-Based)

On va construire une fonction qui :
1. Trouve les voisins les plus proches d’un utilisateur
2. Agrège leurs notes
3. Exclut les films déjà vus
4. Propose un TOP N recommandations

### Films déjà vus par l'utilisateur cible

In [12]:
def get_seen_movies(user_id):
    return df.loc[user_id][df.loc[user_id] > 0].index.tolist()

### Trouver les voisins via KNN

In [13]:
def get_user_neighbors(user_id, df, clf, n_neighbors=6): 
    user_index = df.index.get_loc(user_id)
    distances, indices = clf.kneighbors(df.iloc[user_index, :].values.reshape(1, -1), n_neighbors=n_neighbors)
    neighbor_indices = indices.flatten()[1:] # Exclure l'utilisateur lui-même
    neighbor_user_ids = df.index[neighbor_indices] 
    return neighbor_user_ids, distances.flatten()[1:] # Exclure la distance de l'utilisateur lui-même

### Calculer les recommandations (moyenne des notes des voisins)

On prend les films que les voisins ont notés, on calcule la moyenne, et on exclut les films que l’utilisateur a déjà vus.

In [14]:
def recommend_movies(user_id, df, df_with_nan, clf, n_neighbors=6, N=5): 
    # Films déjà vus
    seen_movies = set(get_seen_movies(user_id))
    
    # Voisins
    neighbor_user_ids, distances = get_user_neighbors(user_id, df, clf, n_neighbors)
    
    neighbor_ratings = df_with_nan.loc[neighbor_user_ids]
    mean_ratings = neighbor_ratings.mean(axis=0)

    mean_ratings = mean_ratings.drop(labels=seen_movies, errors='ignore')

    top_recommendations = mean_ratings.sort_values(ascending=False).head(N)
    return top_recommendations

On teste notre fonction de recommandation pour l’utilisateur 42 :

In [15]:
recommend_movies(42, df, df_with_nan, clf, 6, 10)

/Users/bilal/Documents/Projects/ML_Pro_projects/.venv/lib/python3.14/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


title
12 Angry Men (1957)                          5.0
As Good As It Gets (1997)                    5.0
Apostle, The (1997)                          5.0
Tomorrow Never Dies (1997)                   5.0
Star Trek: The Motion Picture (1979)         5.0
Ridicule (1996)                              5.0
Thin Line Between Love and Hate, A (1996)    5.0
Junior (1994)                                5.0
Desperate Measures (1998)                    5.0
U Turn (1997)                                5.0
dtype: float64


# Au final

Nous avons construit un système de recommandation **User-Based KNN** pour les films. Pour cela, nous avons fusionné les notes et les titres des films, créé une matrice utilisateur × film, puis entraîné un modèle KNN pour identifier les utilisateurs aux goûts similaires. Les recommandations sont produites en agrégeant les notes des voisins tout en excluant les films déjà vus par l'utilisateur.

Le modèle KNN a été appliqué dans l'espace des utilisateurs pour trouver des profils similaires, permettant ainsi de recommander des films basés sur les préférences collectives des utilisateurs proches.